In [11]:
import cv2
import numpy as np
from skimage import feature
import numpy as np
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2 
from skimage.feature import local_binary_pattern
from skimage import feature
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics.pairwise import euclidean_distances
import pyswarms as ps
from pyswarms.discrete import BinaryPSO


In [12]:
# load dataset
with open('X-Train.npy', 'rb') as f:
    X_train = np.load(f)
    
with open('Y-Train.npy', 'rb') as f:
    Y_train = np.load(f)
    
with open('X-Test.npy', 'rb') as f:
    X_test = np.load(f)
    
with open('Y-Test.npy', 'rb') as f:
    Y_test = np.load(f)
    
with open('GA-LBP-Positions.npy', 'rb') as f:
    ga_pos = np.load(f)

with open('PSO-LBP-Positions.npy', 'rb') as f:
    pso_pos = np.load(f)

with open('ACO-LBP-Positions(1).npy', 'rb') as f:
    aco_pos = np.load(f)
   
Positions = [ga_pos,pso_pos,aco_pos,np.array([1]*256)]
Names = ["Genetic Algorithm","Particle Swarm Optimization","Ant Colony Optimization","Actual"]


In [13]:
# Define objective function for each particle
def ParticleFitness(m):
    
    k = list(map(str,list(m)))
    k = int("".join(k),2)

    classifier = KNeighborsClassifier(n_neighbors=k,metric="cosine")
    total_features = 6    
    classifier.fit(_X_train, Y_train)
    P = classifier.score(_X_test,Y_test)
    
    return 1-P

def Fitness(x, alpha=0.88,**k):
    
    file = open('PSO-KNN-Report.txt', 'a')
    n_particles = x.shape[0]
    result = []
    for i in range(n_particles):
        k = list(map(str,list(x[i])))
        k = int("".join(k),2)
        file.write(str(k)+", ")
        if k == 0:
            result.append(1)
        else:
            result.append(ParticleFitness(x[i]))
    file.write("\n")
    return np.array(result)


In [14]:
options = {'c1': 1.4, 'c2': 1.4, 'w':0.8, 'k': 25, 'p':2}
dimensions = 6 

for index  in range(len(Positions)):
    print("Optimizing K value in K-NN with {0} features".format(Names[index]))
    _X_train = X_train[:,Positions[index]==1]
    _X_test = X_test[:,Positions[index]==1]
    
    optimizer = ps.discrete.BinaryPSO(n_particles=25, dimensions=dimensions, options=options)
    cost, pos = optimizer.optimize(Fitness,iters=75, verbose=2)
    optimizer.reset()
    k = list(map(str,list(pos)))
    k = int("".join(k),2)
    
    classifier = KNeighborsClassifier(n_neighbors=k,metric="cosine")
    classifier.fit(_X_train, Y_train)
    print("Best K found is: {0} and its score is {1}".format(k,classifier.score(_X_test,Y_test)))


2022-02-15 19:05:35,076 - pyswarms.discrete.binary - INFO - Optimize for 75 iters with {'c1': 1.4, 'c2': 1.4, 'w': 0.8, 'k': 25, 'p': 2}


Optimizing K value in K-NN with Genetic Algorithm features


pyswarms.discrete.binary: 100%|█████████████████████████████████████████████████████████████████|75/75, best_cost=0.134
2022-02-15 19:05:44,816 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.13414634146341464, best pos: [0 0 0 0 0 1]
2022-02-15 19:05:44,835 - pyswarms.discrete.binary - INFO - Optimize for 75 iters with {'c1': 1.4, 'c2': 1.4, 'w': 0.8, 'k': 25, 'p': 2}


Best K found is: 1 and its score is 0.8658536585365854
Optimizing K value in K-NN with Particle Swarm Optimization features


pyswarms.discrete.binary: 100%|████████████████████████████████████████████████████████████████|75/75, best_cost=0.0976
2022-02-15 19:05:56,809 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.09756097560975607, best pos: [0 0 0 0 1 1]
2022-02-15 19:05:56,823 - pyswarms.discrete.binary - INFO - Optimize for 75 iters with {'c1': 1.4, 'c2': 1.4, 'w': 0.8, 'k': 25, 'p': 2}


Best K found is: 3 and its score is 0.9024390243902439
Optimizing K value in K-NN with Ant Colony Optimization features


pyswarms.discrete.binary: 100%|█████████████████████████████████████████████████████████████████|75/75, best_cost=0.122
2022-02-15 19:06:05,899 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.12195121951219512, best pos: [0 0 0 0 1 1]
2022-02-15 19:06:05,913 - pyswarms.discrete.binary - INFO - Optimize for 75 iters with {'c1': 1.4, 'c2': 1.4, 'w': 0.8, 'k': 25, 'p': 2}


Best K found is: 3 and its score is 0.8780487804878049
Optimizing K value in K-NN with Actual features


pyswarms.discrete.binary: 100%|████████████████████████████████████████████████████████████████|75/75, best_cost=0.0854
2022-02-15 19:06:17,553 - pyswarms.discrete.binary - INFO - Optimization finished | best cost: 0.08536585365853655, best pos: [0 0 0 0 0 1]


Best K found is: 1 and its score is 0.9146341463414634
